# 10个范例带你入门LangChain 

5，数据库问答(Querying Tabular Data): 从数据库/类数据库内容中抽取数据信息。

In [1]:
# 使用你自己的OpenAI API key
openai_api_key='sk-PFwMvMFcU7VXaL5cYM9vT3B'
zhipu_api_key='08132b620644081b192567639fc0a9b1.OAz1XI2d21zLZDk9'

In [2]:
import time
import jwt
 
def generate_token(apikey: str, exp_seconds: int):
    try:
        id, secret = apikey.split(".")
    except Exception as e:
        raise Exception("invalid apikey", e)
 
    payload = {
        "api_key": id,
        "exp": int(round(time.time() * 1000)) + exp_seconds * 1000,
        "timestamp": int(round(time.time() * 1000)),
    }
 
    return jwt.encode(
        payload,
        secret,
        algorithm="HS256",
        headers={"alg": "HS256", "sign_type": "SIGN"},
    )

In [3]:
# from langchain_community.chat_models import ChatOpenAI
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
llm =ChatOpenAI(
        model_name="glm-4",
        openai_api_base="https://open.bigmodel.cn/api/paas/v4",
        openai_api_key=generate_token(zhipu_api_key,10000),
        streaming=False,
        temperature=0.5
    )

## 五，数据库问答(Querying Tabular Data)

In [4]:
# 使用自然语言查询一个 SQLite 数据库，我们将使用旧金山树木数据集
# Don't run following code if you don't run sqlite and follow db
from langchain import OpenAI, SQLDatabase
# %pip install  --user langchain_experimental
from langchain_experimental.sql import SQLDatabaseChain
# llm = OpenAI(temperature=0, openai_api_key=openai_api_key)

In [5]:
sqlite_db_path = 'San_Francisco_Trees.db'
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

In [6]:
db_chain = SQLDatabaseChain.from_llm(llm=llm, db=db, verbose=True)

In [7]:
db_chain.invoke("How many Species of trees are there in San Francisco?")



> Entering new SQLDatabaseChain chain...
How many Species of trees are there in San Francisco?
SQLQuery:SELECT COUNT(DISTINCT "qSpecies") FROM "SFTrees"
SQLResult:

OperationalError: (sqlite3.OperationalError) unrecognized token: ":"
[SQL: SELECT COUNT(DISTINCT "qSpecies") FROM "SFTrees"
SQLResult:]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

1. Find which table to use
2. Find which column to use
3. Construct the correct sql query
4. Execute that query
5. Get the result
6. Return a natural language reponse back

confirm LLM result via pandas

In [ ]:
import sqlite3
%pip install pandas --user
import pandas as pd

# Connect to the SQLite database
connection = sqlite3.connect(sqlite_db_path)

# Define your SQL query
query = "SELECT count(distinct qSpecies) FROM SFTrees"

# Read the SQL query into a Pandas DataFrame
df = pd.read_sql_query(query, connection)

# Close the connection
connection.close()

  Using cached https://files.pythonhosted.org/packages/78/a8/07dd10f90ca915ed914853cd57f79bfc22e1ef4384ab56cb4336d2fc1f2a/pandas-2.0.3-cp38-cp38-macosx_10_9_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d5/fb/a79efcab32b8a1f1ddca7f35109a50e4a80d42ac1c9187ab46522b2407d7/tzdata-2023.3-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/32/4d/aaf7eff5deb402fd9a24a1449a8119f00d74ae9c2efa79f8ef9994261fc2/pytz-2023.3.post1-py2.py3-none-any.whl
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Display the result in the first column first cell
print(df.iloc[0,0])

578
